In [19]:
import csv
import os
import requests
import re

In [20]:
# URL glavne strani
igralci_frontpage_url = 'https://www.basketball-reference.com/leagues/NBA_2023_per_game.html'
# mapa, v katero bomo shranili podatke
igralci_directory = 'podatki'
# ime datoteke v katero bomo shranili glavno stran
frontpage_filename = 'igralci.html'
# ime CSV datoteke v katero bomo shranili podatke
csv_filename = 'igralci_url.csv'

In [21]:
print(csv_filename)

igralci_url.csv


1.Pridobivanje podatkov

In [22]:
import traceback  # za popolno poročilo o napaki


def download_url_to_string(url):
    """Funkcija kot argument sprejme niz in poskusi vrniti vsebino te spletne
    strani kot niz. V primeru, da med izvajanje pride do napake vrne None.
    """
    try:
        # del kode, ki morda sproži napako
        page_content = requests.get(url)
        if page_content.status_code == 200:
            return page_content.text
        else:
            raise ValueError(f"Čudna koda: {page_content.status_code}")
    except Exception:
        # koda, ki se izvede pri napaki
        # dovolj je če izpišemo opozorilo in prekinemo izvajanje funkcije
        print(f"Prišlo je do spodnje napake:\n{traceback.format_exc()}")


def save_string_to_file(text, directory, filename):
    """Funkcija zapiše vrednost parametra "text" v novo ustvarjeno datoteko
    locirano v "directory"/"filename", ali povozi obstoječo. V primeru, da je
    niz "directory" prazen datoteko ustvari v trenutni mapi.
    """
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    with open(path, 'w', encoding='utf-8') as file_out:
        file_out.write(text)
    return None

In [23]:
def save_frontpage(page, directory, filename):
    """Funkcija shrani vsebino spletne strani na naslovu "page" v datoteko
    "directory"/"filename"."""
    html_strani = download_url_to_string(page)
    save_string_to_file(html_strani, directory, filename)

2.Obdelava podatkov

In [24]:
def read_file_to_string(directory, filename):
    #sestavimo pot do datoteke
    file_path = os.path.join(directory, filename) # => "directory, filename"
    #odpremo datoteko za branje
    with open(file_path, "r", encoding = "utf-8") as file:
        #preberemo datoteko (v en niz)
        return file.read()       

In [25]:
save_frontpage(igralci_frontpage_url, igralci_directory, frontpage_filename)

In [26]:
#izluščimo posamezne igralce
def izlusci_igralce(page_content):
    vzorec = r'<tr class="full_table" ><th scope="row" class="right " data-stat="ranker" csk=".*?</tr>'
    return re.findall(vzorec, page_content, re.DOTALL)


In [27]:
print(os.listdir())
text = read_file_to_string(igralci_directory, frontpage_filename)
igralci = izlusci_igralce(text)
print(len(igralci))

['.git', 'Analiza-podatkov.ipynb', 'igralci_url.csv', 'obdelani_podatki', 'podatki', 'README.md', 'venv', 'Zajem-podatkov.ipynb']
539


In [28]:
#izluščimo lastnosti posameznega igralca
def get_players_from_block(block):
    vzorec_ime = r'data-stat="player" csk=".+?,(.+?)"'
    vzorec_priimek = r'data-stat="player" csk="(.*?),.+?"'
    vzorec_starost = r'data-stat="age" >(.*?)</td>' 
    vzorec_pozicija = r'data-stat="pos" >(.*?)</td>' 
    vzorec_stevilo_iger = r'data-stat="g" >(.*?)</td>'
    vzorec_povprecje_tocke = r'data-stat="pts_per_g" >(.*?)</td>'
    vzorec_povprecje_osebne_napake = r'data-stat="pf_per_g" >(.*?)</td>' 
    vzorec_povprecje_asistence = r'data-stat="ast_per_g" >(.*?)</td>' 
    vzorec_povprecje_skoki = r'data-stat="trb_per_g" >(.*?)</td>'
    vzorec_povprecje_trojke = r'data-stat="fg3_per_g" >(.*?)</td>'
    
    
    ime = re.search(vzorec_ime, block).group(1)
    priimek = re.search(vzorec_priimek, block).group(1)
    starost = re.search(vzorec_starost, block, flags=re.DOTALL).group(1)
    pozicija = re.search(vzorec_pozicija, block, flags=re.DOTALL).group(1)
    stevilo_iger= re.search(vzorec_stevilo_iger, block, flags=re.DOTALL).group(1)
    povprecje_tocke = re.search(vzorec_povprecje_tocke, block, flags=re.DOTALL).group(1)
    povprecje_osebne_napake = re.search(vzorec_povprecje_osebne_napake, block, flags=re.DOTALL).group(1)
    povprecje_asistence = re.search(vzorec_povprecje_asistence, block, flags=re.DOTALL).group(1)
    povprecje_skoki = re.search(vzorec_povprecje_skoki, block, flags=re.DOTALL).group(1)
    povprecje_trojke = re.search(vzorec_povprecje_trojke, block, flags=re.DOTALL).group(1)

    return {"ime": ime, 
            "priimek": priimek, 
            "starost": starost, 
            "pozicija": pozicija, 
            "stevilo_iger": stevilo_iger, 
            "povprecje_tocke": povprecje_tocke, 
            "povprecje_osebne_napake": povprecje_osebne_napake, 
            "povprecje_asistence": povprecje_asistence, 
            "povprecje_skoki": povprecje_skoki, 
            "povprecje_trojke": povprecje_trojke}



In [29]:
for igralec in igralci:
    print(get_players_from_block(igralec))

{'ime': 'Precious', 'priimek': 'Achiuwa', 'starost': '23', 'pozicija': 'C', 'stevilo_iger': '55', 'povprecje_tocke': '9.2', 'povprecje_osebne_napake': '1.9', 'povprecje_asistence': '0.9', 'povprecje_skoki': '6.0', 'povprecje_trojke': '0.5'}
{'ime': 'Steven', 'priimek': 'Adams', 'starost': '29', 'pozicija': 'C', 'stevilo_iger': '42', 'povprecje_tocke': '8.6', 'povprecje_osebne_napake': '2.3', 'povprecje_asistence': '2.3', 'povprecje_skoki': '11.5', 'povprecje_trojke': '0.0'}
{'ime': 'Bam', 'priimek': 'Adebayo', 'starost': '25', 'pozicija': 'C', 'stevilo_iger': '75', 'povprecje_tocke': '20.4', 'povprecje_osebne_napake': '2.8', 'povprecje_asistence': '3.2', 'povprecje_skoki': '9.2', 'povprecje_trojke': '0.0'}
{'ime': 'Ochai', 'priimek': 'Agbaji', 'starost': '22', 'pozicija': 'SG', 'stevilo_iger': '59', 'povprecje_tocke': '7.9', 'povprecje_osebne_napake': '1.7', 'povprecje_asistence': '1.1', 'povprecje_skoki': '2.1', 'povprecje_trojke': '1.4'}
{'ime': 'Santi', 'priimek': 'Aldama', 'starost

In [30]:
def players_from_file(filename, directory):
    """Funkcija prebere podatke v datoteki "directory"/"filename" in jih
    pretvori v pripadajoč seznam slovarjev za vsakega igralca posebej."""
    vsebina = read_file_to_string(directory, filename)
    igralci = izlusci_igralce(vsebina)
    slovarji = []
    for igralec in igralci:
        slovarji.append(get_players_from_block(igralec))
    return slovarji

In [31]:
players_from_file(frontpage_filename, igralci_directory)

[{'ime': 'Precious',
  'priimek': 'Achiuwa',
  'starost': '23',
  'pozicija': 'C',
  'stevilo_iger': '55',
  'povprecje_tocke': '9.2',
  'povprecje_osebne_napake': '1.9',
  'povprecje_asistence': '0.9',
  'povprecje_skoki': '6.0',
  'povprecje_trojke': '0.5'},
 {'ime': 'Steven',
  'priimek': 'Adams',
  'starost': '29',
  'pozicija': 'C',
  'stevilo_iger': '42',
  'povprecje_tocke': '8.6',
  'povprecje_osebne_napake': '2.3',
  'povprecje_asistence': '2.3',
  'povprecje_skoki': '11.5',
  'povprecje_trojke': '0.0'},
 {'ime': 'Bam',
  'priimek': 'Adebayo',
  'starost': '25',
  'pozicija': 'C',
  'stevilo_iger': '75',
  'povprecje_tocke': '20.4',
  'povprecje_osebne_napake': '2.8',
  'povprecje_asistence': '3.2',
  'povprecje_skoki': '9.2',
  'povprecje_trojke': '0.0'},
 {'ime': 'Ochai',
  'priimek': 'Agbaji',
  'starost': '22',
  'pozicija': 'SG',
  'stevilo_iger': '59',
  'povprecje_tocke': '7.9',
  'povprecje_osebne_napake': '1.7',
  'povprecje_asistence': '1.1',
  'povprecje_skoki': '2.

3.Shranjevanje podatkov

In [32]:
def write_csv(fieldnames, rows, directory, filename):
    """
    Funkcija v csv datoteko podano s parametroma "directory"/"filename" zapiše
    vrednosti v parametru "rows" pripadajoče ključem podanim v "fieldnames"
    """
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, filename)
    # ko odpremo datoteko, podamo neobevzni argument newline in ga nastavimo na prazen niz,
    # sicer bomo na windowsih imeli grd csv, kjer bo vsaki dejanski vrstici sledila prazna
    with open(path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for row in rows:
            writer.writerow(row)
    return

In [33]:
def write_players_to_csv(players, directory, filename):
    """Funkcija vse podatke iz parametra "ads" zapiše v csv datoteko podano s
    parametroma "directory"/"filename". Funkcija predpostavi, da so ključi vseh
    slovarjev parametra ads enaki in je seznam ads neprazen."""
    # Stavek assert preveri da zahteva velja
    # Če drži se program normalno izvaja, drugače pa sproži napako
    # Prednost je v tem, da ga lahko pod določenimi pogoji izklopimo v
    # produkcijskem okolju
    assert players and (all(slovar.keys() == players[0].keys() for slovar in players))
    imena_stolpcev = (players[0])
    write_csv(imena_stolpcev, players, directory, filename)

In [34]:
vsi_slovarji = players_from_file(frontpage_filename, igralci_directory)
write_players_to_csv(vsi_slovarji, "obdelani_podatki", csv_filename)

4.Vse skupaj

In [35]:
def main(redownload=True, reparse=True):
    """Funkcija izvede celoten del pridobivanja podatkov:
    1. Prenese podatke o igralcih iz spletne strani
    2. Lokalno html datoteko pretvori v lepšo predstavitev podatkov
    3. Podatke shrani v csv datoteko
    """
    # 1. Najprej v lokalno datoteko shranimo glavno stran
    pot_html = os.path.join(igralci_directory, frontpage_filename)
    if redownload or not os.path.exists(pot_html):
        # če zahtevamo ponovno nalaganje ali pa html_datoteka
        # še ne obstaja, jo naložimo s spleta
        save_frontpage(igralci_frontpage_url, igralci_directory, frontpage_filename)
    else:
        print(f"Datoteka {pot_html} že obstaja")
    csv_mapa = "obdelani_podatki"
    pot_csv = os.path.join(csv_mapa, csv_filename)
    # 2. in 3.: obdelava html-ja in shranjevanje v csv
    if reparse or not os.path.exists(pot_csv):
        # če zahtevamo ponovno obdelavo ali pa csv_datoteka
        # še ne obstaja, jo ustvarimo (ponovno)
        vsi_slovarji = players_from_file(frontpage_filename, igralci_directory)
        write_players_to_csv(vsi_slovarji, "obdelani_podatki", csv_filename)
    else:
        print(f"Datoteka {pot_csv} že obstaja")


In [36]:
main(False, False)

Datoteka podatki\igralci.html že obstaja
Datoteka obdelani_podatki\igralci_url.csv že obstaja
